## Логика игры

Приветствие и правила игры

Показ игровой доски ИИ (все в нулях)

Создание рандомных кораблей на доске ИИ

Запрос создания кораблей Игрока (показ при каждом размещении кораблей)

### Loop:
Ход игрока

Проверка правильности хода

Размещение на доске ИИ и отображение его доски

Проверка на ПОБЕДУ

Ход ИИ и отображение доски Игрока

Проверка на ПОБЕДУ


# ...Error Classes

In [513]:
class BoardOutError(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return f"{self.value} лежит за пределами игрового поля. Введите двузначное число, каждая цифра которого от 1 до 6, включительно."


class NonEmptyError(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return f"Выбраная ячейка - {self.value} - непустая. Стрелять можно только в пустые ячейки."


class ShipLocationError(Exception):
    def __init__(self, length, front, direction):
        self.length = length
        self.front = front
        self.direction = direction

    def __str__(self):
        self.front = (self.front[0] + 1, self.front[1] + 1)
        direction_word = {
            1: 'Вправо',
            2: 'Вниз'
        }
        return f"Указанные параметры - длина ({self.length}), положение носа корабля {self.front} и направление ({direction_word[self.direction]}) не позволяют корректно разместить корабль на игровом поле."

# ...Board, Ship, Dot Classes

In [514]:
class Dot:
    def __init__(self, row, col):
        self.row = row
        self.col = col

    def __eq__(self, other):
        return self.row == other[0] and self.col == other[1]

    @property
    def show(self):
        return self.row, self.col
    
    
class Ship:
    def __init__(self, length, front, direction, life):
        self.length = length
        self.front = front
        self.direction = direction
        self.life = life

    def dots(self):
        ship_cells = []
        direction_vector = {
            1: (0, 1),
            2: (1, 0)
        }

        for i in range(self.length):
            cell = (self.front[0] + direction_vector[self.direction][0] * (i), \
                    self.front[1] + direction_vector[self.direction][1] * (i))
            ship_cells.append(cell)
        return ship_cells 


In [515]:
class Board:
    POSSIBLE_CELLS = []
    for i in range(6):
        for j in range(6):
            POSSIBLE_CELLS.append((i, j))
    #     print(POSSIBLE_CELLS)

    @property
    def player_shots_left(self):
        player_shots_left = []
        for i in self.POSSIBLE_CELLS:
            player_shots_left.append(Dot(*i))
        return player_shots_left

    start_cells_conditions = [['o' for i in range(6)] for j in range(6)]

    __s = u'\u220E'  # код квадрата в системе unicode

    def __init__(self, hid, cells_conditions=start_cells_conditions, ships = [], alive_ships = 7):
        self.cells_conditions = cells_conditions  # list
        self.ships = ships  # list with all ships
        self.hid = hid  # bool
        self.alive_ships = alive_ships  # int

    def add_ship(self, ship):  # ставит корабль на доску (если ставить не получается, выбрасываем исключения)
        for i in range(ship.length):
            if ship.dots()[i] in self.contoured_ships() or ship.dots()[i] not in self.POSSIBLE_CELLS:
                raise ShipLocationError(ship.length, ship.front, ship.direction)
        self.ships.append(ship.dots())
        print(f'ships = {self.ships}')
        
        for i in (self.contoured_ships()):
            try:
                self.player_shots_left.remove(i)
            except Exception:
                pass


    def contoured_ships(self):  # точки всех кораблей + их контуров

        existing_ships_dots = []
        for i in range(len(self.ships)):
            existing_ships_dots += self.ships[i]

        contoured_ships_dots = []
        for dot in range(len(existing_ships_dots)):
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    tested_dot = (existing_ships_dots[dot][0] + i, existing_ships_dots[dot][1] + j)
                    if tested_dot in self.POSSIBLE_CELLS:
                        contoured_ships_dots.append(tested_dot)
        contoured_ships_dots = list(set(contoured_ships_dots))
        
        
        return contoured_ships_dots

    def show_board(self):  # выводит доску в консоль в зависимости от параметра hid.
        ships_gui = self.start_cells_conditions.copy()

        existing_ships_dots = []
        for i in range(len(self.ships)):
            existing_ships_dots += self.ships[i]
        
        for i in existing_ships_dots:
            ships_gui[i[0]][i[1]] = self.__s

        if not self.hid:
            return self.gui_print(ships_gui)

    def out(self, dot):  # для точки (объекта класса Dot) возвращает True, если точка выходит за пределы поля
        print(111)
        if dot in self.POSSIBLE_CELLS:
            return False
        else:
            return True

    def shot(self, row, col):  # делает выстрел по доске (если есть попытка выстрелить за \
        # пределы и в использованную точку, нужно выбрасывать исключения
        while True:  # Loop для получения правильного ввода от Игрока

            #             try:
            #                 player_shot = int(input("В какую ячейку будем стрелять?"))
            #                 row = player_shot // 10 - 1
            #                 col = player_shot % 10 - 1
            #             except Exception:
            #                 print('Необходимо ввести двузначное число!')
            #                 break

            if self.out((row, col)):
                raise BoardOutError((row + 1, col + 1))
            else:
                shot = Dot(row, col)

                if shot not in self.player_shots_left:
                    err = NonEmptyError((row + 1, col + 1))
                    print(err)
                else:
                    existing_ships_dots = []
                    for i in range(len(self.ships)):
                        existing_ships_dots += self.ships[i]
                    if (row, col) in existing_ships_dots:
                        self.cells_conditions[row][col] = 'X'
                        self.player_shots_left.remove(Dot(row, col))
                        print(self.cells_conditions)
                    else:
                        self.cells_conditions[row][col] = 'T'
                        self.player_shots_left.remove(Dot(row, col))
                        print(self.cells_conditions)
                    print('Ход компьютера:')
                    break

    def gui_print(self, L):
        print(*[" ", *[i for i in range(1, 7)]], sep=' | ')
        for i in range(6):
            print(i + 1, *L[i], sep=' | ')
        return

# ВНЕШНЯЯ ЛОГИКА

In [516]:
                                                                                # PLAYER CLASS
from abc import ABC
from abc import abstractmethod

class Player(ABC):
    def __init__(self, my_board, opponent_board):
        self.my_board = my_board
        self.opponent_board = opponent_board
        super().__init__()
    
    @abstractmethod    
    def ask(self):
        pass
    
    def move(self):
        opponent_board.shot(*self.ask())
        
                                                                                # USER CLASS
class User(Player):
    def ask(self):
        while True:
            try:
                player_shot = int(input("В какую ячейку будем стрелять? "))
                row = player_shot // 10 - 1
                col = player_shot % 10 - 1
                return row, col
            
            except Exception:
                print('Необходимо ввести двузначное число, каждая цифра которого от 1 до 6! ')
                continue

                                                                                    # AI CLASS

from random import randint

class AI(Player):
    def ask(self):
        shot = opponent_board.player_shots_left[randint(0,len(opponent_board.player_shots_left)-1)]
        return shot.show

In [517]:
                                                                        # Создание полей и игроков
user_board = Board(hid = False)
AI_board = Board(hid = True)
user = User(user_board, AI_board)
ai = AI(AI_board, user_board)

In [518]:
from random import randint

class Game:
    def __init__(self, user, user_board, AI, AI_board):
        self.user = user
        self.user_board = user_board
        self.AI = AI
        self.AI_board = AI_board

    def random_board(self):
        while True:
            # размещение 3-клеточного корабля
            while True:
                try:
                    rand_front = self.user_board.player_shots_left[randint(0, len(self.user_board.player_shots_left)-1)].show
                    ship = Ship(3, rand_front, randint(1,2), 3)

                    self.user_board.add_ship(ship)
                    break
                except Exception:
                    print('exception when 3x user')
                    continue
                
            # размещение 2-клеточных кораблей
            trial = 10000
            n = 2
            while True:
                try:
                    rand_front = self.user_board.player_shots_left[randint(0, len(self.user_board.player_shots_left)-1)].show
                    ship = Ship(2, rand_front, randint(1,2), 2)

                    self.user_board.add_ship(ship)
                    n -= 1
                    if n == 0:
                        break
                    else:
                        continue
                        
                except Exception:
                    if trial == 0:
                        break
                    else:
                        trial -= 1
                    continue
                
            if trial == 0:
                self.user_board.ships = []
                print('start over after TWO user')
                continue
            
            # размещение 1-клеточных кораблей
            trial = 10000
            n = 4
            while True:                
                try:                  
                    rand_front = self.user_board.player_shots_left[randint(0, len(self.user_board.player_shots_left)-1)].show
                    ship = Ship(1, rand_front, randint(1,2), 1)

                    self.user_board.add_ship(ship)
                    n -= 1
                    if n == 0:
                        break
                    else:
                        continue
                        
                except Exception:
                    if trial == 0 or n == 0:
                        break
                    else:
                        trial -= 1
                    continue
            
            if trial == 0:
                print('the end reached user')
                trial == 10000
                self.user_board.ships = []
                print('start over user')
                continue
            else:
                break
            
            
            
        while True:
            # размещение 3-клеточного корабля
            while True:
                try:
                    rand_front = self.AI_board.player_shots_left[randint(0, len(self.AI_board.player_shots_left)-1)].show
                    ship = Ship(3, rand_front, randint(1,2), 3)

                    self.AI_board.add_ship(ship)
                    break
                except Exception:
                    print('exception when 3x')
                    continue
                
            # размещение 2-клеточных кораблей
            trial = 10000
            n = 2
            while True:
                try:
                    rand_front = self.AI_board.player_shots_left[randint(0, len(self.AI_board.player_shots_left)-1)].show
                    ship = Ship(2, rand_front, randint(1,2), 2)

                    self.AI_board.add_ship(ship)
                    n -= 1
                    if n == 0:
                        break
                    else:
                        continue
                        
                except Exception:
                    if trial == 0:
                        break
                    else:
                        trial -= 1
                    continue
                
            if trial == 0:
                self.AI_board.ships = []
                print('start over after TWO')
                continue
            
            # размещение 1-клеточных кораблей
            trial = 10000
            n = 4
            while True:                
                try:                  
                    rand_front = self.AI_board.player_shots_left[randint(0, len(self.AI_board.player_shots_left)-1)].show
                    ship = Ship(1, rand_front, randint(1,2), 1)

                    self.AI_board.add_ship(ship)
                    n -= 1
                    if n == 0:
                        break
                    else:
                        continue
                        
                except Exception:
                    if trial == 0 or n == 0:
                        break
                    else:
                        trial -= 1
                    continue
            
            if trial == 0:
                print('the end reached')
                trial == 10000
                self.AI_board.ships = []
                print('start over')
                continue
            else:
                break       
        

    

    def greet(self):
        print('''Приветствую Вас, друг мой!
Перед началом игры позвольте рассказать о ее правилах''')
    
    def loop(self):
        while True:
            user.move()
            if AI_board.alive_ships == 0:
                print('User wins!')
                break
            ai.move()
            if user_board.alive_ships == 0:
                print('AI wins!')
                break
    def start(self):
        self.greet()
        self.loop()


In [519]:
g = Game(user, user_board, AI, AI_board)
g.random_board()

exception when 3x user
ships = [[(3, 3), (3, 4), (3, 5)]]
ships = [[(3, 3), (3, 4), (3, 5)], [(1, 4), (1, 5)]]
ships = [[(3, 3), (3, 4), (3, 5)], [(1, 4), (1, 5)], [(5, 4), (5, 5)]]
ships = [[(3, 3), (3, 4), (3, 5)], [(1, 4), (1, 5)], [(5, 4), (5, 5)], [(1, 1)]]
ships = [[(3, 3), (3, 4), (3, 5)], [(1, 4), (1, 5)], [(5, 4), (5, 5)], [(1, 1)], [(4, 0)]]
ships = [[(3, 3), (3, 4), (3, 5)], [(1, 4), (1, 5)], [(5, 4), (5, 5)], [(1, 1)], [(4, 0)], [(5, 2)]]
the end reached user
start over user
ships = [[(4, 0), (4, 1), (4, 2)]]
ships = [[(4, 0), (4, 1), (4, 2)], [(2, 0), (2, 1)]]
ships = [[(4, 0), (4, 1), (4, 2)], [(2, 0), (2, 1)], [(0, 3), (0, 4)]]
ships = [[(4, 0), (4, 1), (4, 2)], [(2, 0), (2, 1)], [(0, 3), (0, 4)], [(4, 4)]]
ships = [[(4, 0), (4, 1), (4, 2)], [(2, 0), (2, 1)], [(0, 3), (0, 4)], [(4, 4)], [(2, 5)]]
ships = [[(4, 0), (4, 1), (4, 2)], [(2, 0), (2, 1)], [(0, 3), (0, 4)], [(4, 4)], [(2, 5)], [(2, 3)]]
ships = [[(4, 0), (4, 1), (4, 2)], [(2, 0), (2, 1)], [(0, 3), (0, 4)], [(4, 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

...

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception when 3x
exception 

KeyboardInterrupt: 

In [497]:
len(user_board.player_shots_left)

36

In [478]:
len(g.user_board.POSSIBLE_CELLS)

36

In [502]:
user_board.show_board()
AI_board.show_board()

  | 1 | 2 | 3 | 4 | 5 | 6
1 | o | o | o | o | o | o
2 | o | o | o | o | o | o
3 | o | o | o | o | o | o
4 | o | o | o | o | o | o
5 | o | o | o | o | o | o
6 | o | o | o | o | o | o


In [312]:
player_shots_left = []
for i in user_board.POSSIBLE_CELLS:
    player_shots_left.append(Dot(*i))
user_board.ships = []
user_board.start_cells_conditions = [['o' for i in range(6)] for j in range(6)]

In [ ]:
import sys

while True:
    g = Game(user, user_board, AI, AI_board)
    g.start()
    play_again = input('Хотите сыграть еще раз? Если да, то введите 1, если нет, то - любой другой символ')
    if play_again == '1':
        continue
    else:
        sys.exit('До скорой встречи!')    
    


In [ ]:
grid = [['o' for i in range(6)] for i in range(6)]

def print_grid(L):
    print(*[" ", *[i for i in range(1,7)]], sep=' | ')
    for i in range(6):
        print(i+1, *L[i], sep=' | ')
    return

In [ ]:
# Python program showing the use of
# @property
  
class Geeks:
     def __init__(self):
          self._age = 0
       
     # using property decorator
     # a getter function
     @property
     def age(self):
         print("getter method called")
         return self._age
       
     # a setter function
     @age.setter
     def age(self, a):
         if(a < 18):
            err = ValueError("Sorry you age is below eligibility criteria")
            print(err)
         print("setter method called")
         self._age = a
  
mark = Geeks()
print(mark.age)
mark.age = int(input())
  
print(mark.age)

In [524]:
class Test:
    b = 123
    
    def __init__(self, a):
        self.a = self.b
t = Test(10)
# print(t.a)
# print(t.b)
k = Test(11)

10
11
